# PTT 論壇爬蟲

這本jupyter notebook 主要希望做到的幾點
1. 成功的爬到八卦版的最新幾頁的文章加上評論
2. 抓到評論的：噓，→ ，推
3. 做成 pandas Dataframe

Main objectives 
1. scrape post content and comments from PTT
2. get the "attitude" of comments (possible sentiment analysis)
3. df.to_csv() to save as an csv file

In [1]:
# import libraries
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
import re

## Inspecting PTT With Inspector Tools
右鍵按inspect來看網頁的HTML code <br>
我們會看到我們的第一步其實是要抓出一個List的網站鏈接<br>
right click inspect to check code. we need to get a list of urls from div class title

In [2]:
# Note that this web contains a age restriction page, so we need to navigate pass this page with selenium

# set up chrome webdriver
driver = webdriver.Chrome()

# reqeust
url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
driver.get(url)

# click button
driver.find_element_by_class_name('btn-big').click()

# create beautiful soup
pagesource = driver.page_source
soup = BeautifulSoup(pagesource)

# check our soup
# soup

In [3]:
# since the index number of the webpage changes as the number of posts change, lets
# find the index of the previous page to set an index number

# find all buttons
page_navigator = soup.find('div', class_ = 'btn-group btn-group-paging').find_all('a', class_ = 'btn wide')

# we only want the previous page button href text
url_index = page_navigator[1]['href']
url_index  #check

'/bbs/Gossiping/index39092.html'

In [4]:
# now lets get the integer of url_index
url_index = int(re.search(r'\d+', url_index).group())

# the url_index + 1 will be our first page
url_index = int(url_index) + 1
url_index

# okay so our page index will be a list that iterates backwards by a decrement of 1 depending on how many pages you need

39093

In [5]:
# we see that the class title of each section leads to another page that contains the information we need
# right now all we get to see is the titles
# we need to find a way to navigate into all the pages
# or we can create a tone of independent soups
# i will try the latter since im not that great with selenium 

# find the list of class_ = title
post_container = soup.find_all('div', class_ = 'title')
print(len(post_container)) # should expect 9 i our case but note it changes due to page size!

14


In [6]:
# now lets try getting all the urls 

# define a first post
first_post = post_container[0]

# accest first post href
first_post.a['href']

'/bbs/Gossiping/M.1599550604.A.2AF.html'

In [7]:
# Great now we just combine this to a base url
base_url = 'https://www.ptt.cc'
first_post_url = base_url + first_post.a['href']
first_post_url

'https://www.ptt.cc/bbs/Gossiping/M.1599550604.A.2AF.html'

# Scraping Post content
After we got the url on the first page we now scrape info of the first post <br>
for now we ignore all images in the posts

In [8]:
# now since we have a first post url lets try making a request for this url

# set up chrome webdriver
driver = webdriver.Chrome()

# reqeust
driver.get(first_post_url)

# click button
driver.find_element_by_class_name('btn-big').click()

# create beautiful soup
pagesource = driver.page_source
soup = BeautifulSoup(pagesource)

# check our soup
# soup

### Getting Post Main Content

In [9]:
# parsing title under <span> class=article-meta-value
title_container = soup.find_all('div', class_ = 'article-metaline')

# find the one we need which is 1
print(len(title_container))
for i in range(3):
    print(title_container[i])
    
# title that we want
title = title_container[1]
title.text

3
<div class="article-metaline"><span class="article-meta-tag">作者</span><span class="article-meta-value">a98987605 (a)</span></div>
<div class="article-metaline"><span class="article-meta-tag">標題</span><span class="article-meta-value">[問卦] 故宮的東西都是垃圾吧？</span></div>
<div class="article-metaline"><span class="article-meta-tag">時間</span><span class="article-meta-value">Tue Sep  8 15:36:42 2020</span></div>


'標題[問卦] 故宮的東西都是垃圾吧？'

In [10]:
# parsing main content under bbs-scren bbs-content
# the problem with the main content is that the text doesnt have a tag
# we will need to do a lot of processing to get only the content
# However, there is a tag for the section above, hence we can use the next_sibling command to get the text
# and instead of finding the entire body, we find the class'richcontent'

content = soup.find('div', class_ = 'bbs-screen bbs-content')
content = str(content)
print(content)
type(content)   

<div class="bbs-screen bbs-content" id="main-content"><div class="article-metaline"><span class="article-meta-tag">作者</span><span class="article-meta-value">a98987605 (a)</span></div><div class="article-metaline-right"><span class="article-meta-tag">看板</span><span class="article-meta-value">Gossiping</span></div><div class="article-metaline"><span class="article-meta-tag">標題</span><span class="article-meta-value">[問卦] 故宮的東西都是垃圾吧？</span></div><div class="article-metaline"><span class="article-meta-tag">時間</span><span class="article-meta-value">Tue Sep  8 15:36:42 2020</span></div>
那個啊
故宮的東西
大部分都是從支那過來的，跟垃圾沒兩樣吧？
中國 = 垃圾 懂？

拿去資源回收或是某些人想要就還他們吧
幹嘛要花稅金保存跟展示這些垃圾呀？

有沒有卦？

--
<span class="f2">※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 101.10.109.78 (臺灣)
</span><span class="f2">※ 文章網址: <a href="https://www.ptt.cc/bbs/Gossiping/M.1599550604.A.2AF.html" rel="nofollow" target="_blank">https://www.ptt.cc/bbs/Gossiping/M.1599550604.A.2AF.html</a>
</span></div>


str

In [11]:
import re 
content = re.findall(r'[\u4e00-\u9fff]+', content)
type(content[1])

str

In [12]:
# parse comments
# comments have identical class called "push" hence we use find all
comments_container = soup.find_all('div', class_ = 'push')
print(len(comments_container)) # should have 11 comments in this case

0


In [13]:
# get the first comment
# if there is an error here it means there are no comments (which is very likely as it alwaays scrapes the newest post first)
# so we have to add an if statement for this later when we combine everything
first_comment = comments_container[0]
first_comment_type = first_comment.span.text 
first_comment_context = first_comment.find('span', class_ = 'f3 push-content').text
first_comment_context

IndexError: list index out of range

# Combining Everything with Loops

Our next step will to try and loop everything together so we can get a larget amount of posts at once <br>
for this notebook, we will simply gather all txt information (both post context and comments)<br>
this can be used to make word clouds for social listening

In [17]:
# lets see what we need 
#     1. the outer laywer will be loop for different pages (indices)
#     2. inner should have a loop for 
#         - different posts url
#         - post comments

# lets start by declaring our single list variable that will contain all texts of our web scrapping process
mylist = []

# we want to monitor the loop in to check if we are accessing the page at a human-like rate
start_time = time()
requests = 0

# set up chrome webdriver
driver = webdriver.Chrome()

# reqeust
url = 'https://www.ptt.cc/bbs/Gossiping/index.html'  # can be changed to other ptt sections
driver.get(url)

# click age restriction button
driver.find_element_by_class_name('btn-big').click()

# create beautiful soup
pagesource = driver.page_source
soup = BeautifulSoup(pagesource)

# find url index
page_navigator = soup.find('div', class_ = 'btn-group btn-group-paging').find_all('a', class_ = 'btn wide')
url_index = page_navigator[1]['href']
url_index = int(re.search(r'\d+', url_index).group())
url_index = int(url_index) + 1   # the number that should be in our url

# Create list
index_list = []
for i in range(10):       # 100 pages worth of data
    index_list.append(url_index)
    url_index = url_index - 1

# first for loop for index_list
for i in index_list:
    
    # next we want to get a list of urls after we access each page
    # first we create a new url from the for loop
    url = 'https://www.ptt.cc/bbs/Gossiping/index' + str(i) + '.html'  
    driver.get(url)
    
    # temporarily pause loop with sleep function
    sleep(randint(8,15))
    
    # create beautiful soup
    pagesource = driver.page_source
    soup = BeautifulSoup(pagesource)
    
    # Create a post container
    post_container = soup.find_all('div', class_ = 'title')
    
    # create a list for post urls on certain page
    posts_url = []
    for i in range(len(post_container)):
        if post_container[i].find('a') is not None:
            posts_url.append(str(post_container[i].a['href']))
        
    # Now we create a loop that accesses all posts
    for i in posts_url:
        
        # create a new url here
        url = 'https://www.ptt.cc' + i
        driver.get(url)                            # repeate everything again.....

        ###################################### Monitor by pausing and calculating time
        
        # temporarily pause loop with sleep function
        sleep(randint(8,15))

        # monitor requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request: {}; Frequency: {} request/s'.format(requests, requests/elapsed_time))
        clear_output(wait=True)
    
        #########################################   Soup
        
        # create beautiful soup
        pagesource = driver.page_source
        soup = BeautifulSoup(pagesource)
        
        ###################################### Main content
        
        content_all = soup.find('div', class_ = 'bbs-screen bbs-content')    # THIS PART NEEDS IMPROVEMENT
        content_all = str(content_all)
        content_ch = re.findall(r'[\u4e00-\u9fff]+', content_all)  # using regular expression and chinese character unicode we can find all chinese chrarcters only
        mylist.append(content_ch)
        
        ###################################### comments
        
        comments_container = soup.find_all('div', class_ = 'push')
        if len(comments_container) != 0:
            for i in range(len(comments_container)):
                mylist.append(comments_container[i].find('span', class_ = 'f3 push-content').text)
        
    

Request: 184; Frequency: 0.07743006352758633 request/s


In [16]:
mylist

<a href="/bbs/Gossiping/M.1599550186.A.8B5.html">Re: [新聞]歷史老師看三國史:早期國民黨要同情讚美蜀</a>

In [ ]:
# now lets change it to a dataframe
df = pd.DataFrame({
    'text':mylist,
})


# save as excel file

df.to_excel(r"C:\Users\tiger\Desktop\jupyter notebook works\ptt\ptt.xlsx.xlsx")